# Visualização de Queries do Banco de Dados

## Configuração do Ambiente

Célula inicial para importar as bibliotecas necessárias e estabelecer a conexão com o banco de dados DuckDB.

In [1]:
import duckdb
import pandas as pd

# Conecta ao banco de dados persistente
con = duckdb.connect(database='BancoNaoRelacional.db', read_only=True)

# Configura o pandas para exibir todas as colunas
pd.set_option('display.max_columns', None)

---

## Query 1: Relação entre Desempenho no ENEM e Remuneração

**Objetivo:** Analisar a relação entre o desempenho médio dos alunos no ENEM e sua remuneração média após ingressarem no mercado de trabalho.

**Lógica:** A query associa o ano de admissão do aluno no mercado de trabalho (`data_admissao_declarada`) com o ano do ENEM (`nu_ano`), assumindo que ambos representam o mesmo ciclo de formação. A análise é focada em uma escola específica.

In [2]:
query1 = """
WITH medias_salariais AS (
    -- Calcula a média da remuneração nominal e do salário contratual dos alunos por:
    -- 1. ano de admissão no trabalho (extraído de data_admissao_declarada)
    -- 2. escola em que o aluno estudou (via co_escola_educacenso na fato_aluno_trajetoria)
    SELECT
        EXTRACT(YEAR FROM data_admissao_declarada) AS ano_admissao,
        co_escola_educacenso,
        ROUND(AVG(vl_remun_media_nom), 2) AS media_remun_media_nom,
        ROUND(AVG(vl_salario_contratual), 2) AS media_salario_contratual
    FROM
        fato_aluno_trajetoria
    WHERE
        data_admissao_declarada IS NOT NULL
    GROUP BY
        ano_admissao,
        co_escola_educacenso
)

-- Associa as médias salariais calculadas com as médias de desempenho no ENEM 
-- para os mesmos anos e escolas, retornando os indicadores lado a lado.
SELECT
    ms.ano_admissao,
    ms.co_escola_educacenso,
    ms.media_remun_media_nom,
    ms.media_salario_contratual,
    de.nu_media_cn_enem,
    de.nu_media_ch_enem,
    de.nu_media_lp_enem,
    de.nu_media_mt_enem,
    de.nu_media_red_enem
FROM
    medias_salariais ms
JOIN dim_escola_indicadores_anuais de
    ON ms.ano_admissao = de.nu_ano
    AND ms.co_escola_educacenso = de.co_escola_educacenso
WHERE
    ms.co_escola_educacenso = '53006178'  -- IFB (Instituto Federal de Brasília)
ORDER BY
    ms.ano_admissao;
"""

df1 = con.execute(query1).df()
print("Resultado da Query 1:")
display(df1)

Resultado da Query 1:


,ano_admissao,co_escola_educacenso,media_remun_media_nom,media_salario_contratual,nu_media_cn_enem,nu_media_ch_enem,nu_media_lp_enem,nu_media_mt_enem,nu_media_red_enem
0,2010,53006178,6930.35,5778.04,545.00,565.25,497.55,516.75,593.18
1,2011,53006178,5097.44,3552.15,464.68,480.41,505.84,490.82,454.38
2,2012,53006178,4700.22,3575.30,509.25,546.81,497.48,544.22,527.39
3,2013,53006178,4571.02,2986.11,518.91,524.60,494.21,574.02,552.22
4,2014,53006178,4017.11,2959.50,523.88,562.02,513.67,514.62,505.33
5,2015,53006178,3277.85,2298.46,506.86,558.60,499.47,478.62,544.00


---

## Query 2: Correlação entre Salário e Desempenho no ENEM por Município

**Objetivo:** Analisar a correlação entre o salário médio dos egressos e o desempenho médio no ENEM das escolas localizadas no mesmo município do vínculo empregatício.

**Lógica:** A query calcula a média salarial por município e ano de admissão, e a média de notas do ENEM por município e ano. Em seguida, junta os resultados para comparação.

**Correção:** A query original usava colunas `id_loc_vinculo` e `id_loc_escola` que não existem. A versão corrigida junta os dados usando `municipio` e `uf`.

In [3]:
query2 = """
WITH vinculos_por_localizacao AS (
    -- Agrupa os egressos por ano de admissão e localização do vínculo, calculando o salário médio.
    SELECT
        EXTRACT(YEAR FROM data_admissao_declarada) AS ano_admissao,
        uf_vinculo,
        municipio_vinculo,
        AVG(vl_salario_contratual) AS media_salarial_local
    FROM
        fato_aluno_trajetoria
    WHERE
        data_admissao_declarada IS NOT NULL AND municipio_vinculo IS NOT NULL
    GROUP BY
        ano_admissao, uf_vinculo, municipio_vinculo
),
enem_por_localizacao AS (
    -- Calcula a média geral do ENEM para cada escola por ano e localização.
    SELECT
        nu_ano,
        uf_escola,
        municipio_escola,
        -- Calcula a média das 5 notas do ENEM
        AVG((nu_media_cn_enem + nu_media_ch_enem + nu_media_lp_enem + nu_media_mt_enem + nu_media_red_enem) / 5) AS media_geral_enem_escola
    FROM
        dim_escola_indicadores_anuais
    WHERE municipio_escola IS NOT NULL
    GROUP BY
        nu_ano, uf_escola, municipio_escola
)
-- Junta os dados de salários e desempenho no ENEM pela localização e ano.
SELECT
    vpl.ano_admissao,
    vpl.uf_vinculo,
    vpl.municipio_vinculo,
    ROUND(vpl.media_salarial_local, 2) AS media_salarial,
    ROUND(epl.media_geral_enem_escola, 2) AS media_enem
FROM
    vinculos_por_localizacao vpl
JOIN
    enem_por_localizacao epl ON vpl.municipio_vinculo = epl.municipio_escola 
                            AND vpl.uf_vinculo = epl.uf_escola 
                            AND vpl.ano_admissao = epl.nu_ano
ORDER BY
    vpl.uf_vinculo ASC, vpl.municipio_vinculo ASC;
"""

df2 = con.execute(query2).df()
print("Resultado da Query 2:")
display(df2)

Resultado da Query 2:


,ano_admissao,uf_vinculo,municipio_vinculo,media_salarial,media_enem
0,2014,AL,Arapiraca,1305.20,510.58
1,2012,AL,Arapiraca,2200.44,521.72
2,2010,AL,Arapiraca,16813.14,527.62
3,2015,AL,Maceió,45.21,515.70
4,2012,AL,Maceió,8933.59,522.95
...,...,...,...,...,...
229,2009,SP,São Paulo,8622.22,536.35
230,2012,SP,São Paulo,2539.22,563.45
231,2015,TO,Palmas,2180.61,540.51
232,2014,TO,Palmas,3419.89,526.99


---

## Query 3: Características do Mercado de Trabalho vs. Desempenho no ENEM

**Objetivo:** Analisar a relação entre as características do mercado de trabalho local (total de vínculos, horas, remuneração) e o desempenho médio no ENEM das escolas na mesma localidade.

**Lógica:** A query agrega dados de vínculos ativos por localidade e a média do ENEM por localidade, depois os junta para análise.

**Correção:** A query original usava colunas `id_loc_vinculo`, `id_vinculo` e `id_loc_escola` que não existem. A versão corrigida junta os dados usando `municipio` e `uf` e agrupa por eles.

In [4]:
query3 = """
WITH vinculos_por_localizacao AS (
    -- Agrega dados de vínculos empregatícios por localização (município/UF).
    -- Calcula o total de vínculos, média de horas contratuais e remuneração média.
    SELECT
        uf_vinculo,
        municipio_vinculo,
        COUNT(*) AS total_vinculos,
        AVG(qtd_hora_contr) AS media_horas_contratuais,
        AVG(vl_remun_media_nom) AS remuneracao_media_nominal
    FROM
        fato_aluno_trajetoria
    WHERE
        indicador_vinculo_ativo = TRUE
        AND qtd_hora_contr > 0
        AND vl_remun_media_nom > 0
        AND municipio_vinculo IS NOT NULL
    GROUP BY
        uf_vinculo, municipio_vinculo
    HAVING
        COUNT(*) >= 5
),
media_enem_por_localizacao AS (
    -- Calcula a média geral do ENEM e a quantidade de escolas por localização.
    SELECT
        uf_escola,
        municipio_escola,
        AVG((nu_media_cn_enem + nu_media_ch_enem + nu_media_lp_enem + nu_media_mt_enem + nu_media_red_enem) / 5) AS media_enem,
        COUNT(DISTINCT co_escola_educacenso) AS qtd_escolas
    FROM
        dim_escola_indicadores_anuais
    WHERE municipio_escola IS NOT NULL
    GROUP BY
        uf_escola, municipio_escola
)
-- Junta os dados de vínculos e de desempenho no ENEM pela localização.
SELECT
    vpl.uf_vinculo AS uf,
    vpl.municipio_vinculo AS municipio,
    vpl.total_vinculos,
    ROUND(vpl.media_horas_contratuais, 2) AS media_horas_contratuais,
    ROUND(vpl.remuneracao_media_nominal, 2) AS remuneracao_media_nominal,
    -- Calcula o valor médio por hora contratual
    ROUND(vpl.remuneracao_media_nominal / NULLIF(vpl.media_horas_contratuais, 0), 2) AS valor_hora_medio,
    ROUND(mepl.media_enem, 2) AS media_enem,
    mepl.qtd_escolas
FROM
    vinculos_por_localizacao vpl
JOIN
    media_enem_por_localizacao mepl ON vpl.municipio_vinculo = mepl.municipio_escola AND vpl.uf_vinculo = mepl.uf_escola
ORDER BY
    vpl.uf_vinculo, remuneracao_media_nominal DESC;
"""

df3 = con.execute(query3).df()
print("Resultado da Query 3:")
display(df3)

Resultado da Query 3:


,uf,municipio,total_vinculos,media_horas_contratuais,remuneracao_media_nominal,valor_hora_medio,media_enem,qtd_escolas
0,AC,Rio Branco,7,37.43,7934.36,211.99,498.52,38
1,AL,Arapiraca,10,36.00,6689.44,185.82,519.46,29
2,AL,Maceió,29,33.72,5132.86,152.20,516.75,131
3,AM,Manaus,29,38.07,3428.26,90.05,507.03,170
4,AM,Itapiranga,10,40.00,1171.25,29.28,446.21,1
...,...,...,...,...,...,...,...,...
170,SP,Guarulhos,8,38.50,1505.49,39.10,530.97,159
171,SP,Santa Bárbara d'Oeste,11,44.00,1297.42,29.49,521.83,26
172,TO,Paraíso do Tocantins,8,44.00,4959.98,112.73,495.03,9
173,TO,Palmas,56,39.45,3959.55,100.38,529.18,35


---

## Query 4: Persistência no Emprego vs. Taxa de Abandono Escolar

**Objetivo:** Comparar a "persistência no emprego" (tempo de emprego) dos egressos com a "taxa de abandono" da instituição no ano em que ingressaram.

**Lógica:** A query foca em alunos de uma escola específica que ingressaram entre 2009 e 2015. Ela junta os dados do aluno com os indicadores da escola no ano de ingresso para obter a taxa de abandono e, em seguida, categoriza tanto o tempo de emprego quanto a taxa de abandono em faixas.

In [5]:
query4 = """
-- Esta query compara a "persistência no emprego" (tempo de emprego) dos egressos do IFB 
-- com a "taxa de abandono" da instituição no ano em que ingressaram.
-- O objetivo é verificar se há uma correlação entre a evasão escolar e a permanência no primeiro emprego.
-- A análise foca nos alunos que ingressaram entre 2009 e 2015.

WITH dados_analise_completo AS (
    SELECT
        fat.id_aluno,
        fat.ano_entrada_ifb,
        fat.tempo_emprego,
        fat.data_admissao_declarada,
        deia.nu_taxa_abandono,
        ROW_NUMBER() OVER (
            PARTITION BY fat.id_aluno
            ORDER BY fat.data_admissao_declarada DESC  
        ) AS ordem_emprego
    FROM
        fato_aluno_trajetoria fat
    JOIN
        dim_escola_indicadores_anuais deia ON fat.co_escola_educacenso = deia.co_escola_educacenso
                                           AND fat.ano_entrada_ifb = deia.nu_ano
    WHERE
        fat.ano_entrada_ifb BETWEEN 2009 AND 2015
        AND fat.co_escola_educacenso = '53006178'
        AND fat.motivo_desligamento IS NOT NULL AND fat.motivo_desligamento != 'NAO DESLIGADO NO ANO'
        AND fat.tempo_emprego IS NOT NULL
        AND deia.nu_taxa_abandono IS NOT NULL
),

primeiro_emprego AS (
    SELECT *
    FROM dados_analise_completo
    WHERE ordem_emprego = 1
)

SELECT
    id_aluno,
    ano_entrada_ifb,
    CASE
        WHEN tempo_emprego <= 12 THEN 'BAIXA'
        WHEN tempo_emprego > 12 AND tempo_emprego <= 36 THEN 'MEDIA'
        ELSE 'ALTA'
    END AS persistencia_emprego,
    CASE
        WHEN nu_taxa_abandono < 5 THEN 'BAIXA'
        WHEN nu_taxa_abandono >= 5 AND nu_taxa_abandono < 10 THEN 'MEDIA'
        ELSE 'ALTA'
    END AS categoria_abandono_escolar
FROM
    primeiro_emprego
ORDER BY
    id_aluno;
"""

df4 = con.execute(query4).df()
print("Resultado da Query 4:")
display(df4)

Resultado da Query 4:


,id_aluno,ano_entrada_ifb,persistencia_emprego,categoria_abandono_escolar
0,5,2014,BAIXA,ALTA
1,12,2014,ALTA,ALTA
2,14,2013,BAIXA,MEDIA
3,18,2015,BAIXA,BAIXA
4,19,2014,ALTA,ALTA
...,...,...,...,...
3881,26108,2014,MEDIA,ALTA
3882,26138,2012,BAIXA,ALTA
3883,26484,2012,BAIXA,ALTA
3884,26767,2015,BAIXA,BAIXA


---

## Query 5: Mobilidade de Alunos para Emprego

**Objetivo:** Identificar alunos de cursos presenciais que conseguiram um emprego em um município diferente daquele onde estudaram.

**Lógica:** A query filtra por alunos de cursos presenciais com remuneração anual acima de R$1000 e compara o município da escola com o município do vínculo empregatício.

In [6]:
query5 = """
-- Esta query identifica alunos de cursos presenciais que conseguiram um emprego em um município
-- diferente daquele onde estudaram, com uma remuneração anual superior a R$1000.
-- O resultado mostra o aluno, seu curso, a remuneração, e os municípios da escola e do emprego.

WITH dados_analise_completo AS (
    SELECT
        fat.id_aluno,
        fat.vl_ultima_remuneracao_ano AS ultima_remuneracao_anual,
        fat.no_curso,
        fat.modalidade_ensino,
        deia.municipio_escola,
        fat.municipio_vinculo AS municipio_empregado,
        ROW_NUMBER() OVER (
            PARTITION BY fat.id_aluno 
            ORDER BY fat.vl_ultima_remuneracao_ano DESC
        ) AS maior_remuneracao,
        ROW_NUMBER() OVER (
            PARTITION BY fat.id_aluno
            ORDER BY fat.data_admissao_declarada DESC  
        ) AS ordem_emprego   -- Para obter o municipio ao qual o aluno está empregado atualmente
    FROM
        fato_aluno_trajetoria fat
    JOIN
        dim_escola_indicadores_anuais deia ON fat.co_escola_educacenso = deia.co_escola_educacenso
    WHERE
        -- Mantivemos a lógica de filtrar por 'Educação Presencial'.
        fat.modalidade_ensino = 'Educação Presencial'
        AND fat.vl_ultima_remuneracao_ano >= 1000
        AND deia.municipio_escola != fat.municipio_vinculo
        -- Garante que as localidades são válidas para comparação
        AND deia.municipio_escola IS NOT NULL
        AND fat.municipio_vinculo IS NOT NULL
),

ultimo_emprego AS (
    SELECT * 
    FROM dados_analise_completo
    WHERE maior_remuneracao = 1 AND ordem_emprego = 1
)

SELECT id_aluno, ultima_remuneracao_anual, no_curso, modalidade_ensino, municipio_escola, municipio_empregado FROM ultimo_emprego
    ORDER BY        
        ultima_remuneracao_anual DESC;
"""

df5 = con.execute(query5).df()
print("Resultado da Query 5:")
display(df5)

Resultado da Query 5:


,id_aluno,ultima_remuneracao_anual,no_curso,modalidade_ensino,municipio_escola,municipio_empregado
0,7925,47282.95,ORÇAMENTO PÚBLICO,Educação Presencial,Brasília,Curitiba
1,26883,46590.19,FIC ENSINO DE CIÊNCIAS PARA OS ANOS INICIAIS D...,Educação Presencial,Brasília,Santo André
2,17997,38132.64,PROCESSO INTERNO DE LICITAÇÃO,Educação Presencial,Brasília,Uberaba
3,8180,37273.11,TÉCNICO EM AGROPECUÁRIA,Educação Presencial,Brasília,Planaltina
4,1017,32176.84,CIÊNCIA DA COMPUTAÇÃO,Educação Presencial,Brasília,São Paulo
...,...,...,...,...,...,...
2699,17910,1000.00,CAPACITAÇÃO NA NORMA REGULAMENTADORA 10 (NR-10),Educação Presencial,Brasília,Goiânia
2700,20946,1000.00,LICENCIATURA EM EDUCAÇÃO PROFISSIONAL PARA DIP...,Educação Presencial,Brasília,Cuiabá
2701,13427,1000.00,TÉCNICO EM SERVIÇOS PÚBLICOS,Educação Presencial,Brasília,Valparaíso de Goiás
2702,13868,1000.00,AUXILIAR DE MARKETING,Educação Presencial,Brasília,Valparaíso de Goiás


---

## Finalização

Fecha a conexão com o banco de dados.

In [7]:
con.close()